# Main statistical analysis
- narrow candidate to 10 candidate using four metrics(total profit, weekly profit, monthly profit, flights frequency)


In [1]:
import pandas as pd
import numpy as np

roundTrip_profit_g = pd.read_csv('../data/temproary_data/roundTrip_profit.csv')
roundTrip_profit_g['round_trip_route_IATA'] = roundTrip_profit_g['round_trip_route_IATA'].apply(lambda x: eval(x))
roundTrip_profit_g['inbound_FL_DATE'] = pd.to_datetime(roundTrip_profit_g['inbound_FL_DATE'])
roundTrip_profit_g['outbound_FL_DATE'] = pd.to_datetime(roundTrip_profit_g['outbound_FL_DATE'])

roundTrip_fre_g = pd.read_csv('../data/temproary_data/round_trip_flights.csv')
roundTrip_fre_g['round_trip_route_IATA'] = roundTrip_fre_g['round_trip_route_IATA'].apply(lambda x: eval(x))

## Aggregrating all the factors and assign ranks to each factors for round-trip route
consider factors:
- flight frequency
- flight profit (total profit - total cost)
- TODO: weekly profit

In [2]:
# calculate the ranking of the profit
roundTrip_profit = (roundTrip_profit_g.groupby('round_trip_route_IATA')['profit']
                 .sum()
                 .reset_index()
                 .rename(columns={'profit':'round_trip_profit'}))
roundTrip_profit = (roundTrip_profit.assign(round_trip_profit_ranking = roundTrip_profit['round_trip_profit']
                                    .rank(ascending = False, method = 'min')))


# calculate the ranking of the frequency
roundTrip_fre = (roundTrip_fre_g.groupby('round_trip_route_IATA')['outbound_OCCUPANCY_RATE']
                 .count().sort_values(ascending=False)
                 .reset_index()
                 .rename(columns={'outbound_OCCUPANCY_RATE':'round_trip_flights_count'}))
roundTrip_fre = roundTrip_fre[roundTrip_fre['round_trip_route_IATA'].isin(roundTrip_profit['round_trip_route_IATA'])]
roundTrip_fre = (roundTrip_fre.assign(round_trip_flights_count_ranking = roundTrip_fre['round_trip_flights_count']
                              .rank(ascending = False, method = 'min')))

# calculate the monthly average profit
roundTrip_profit_m = (roundTrip_profit_g.assign(month = np.minimum(roundTrip_profit_g['inbound_FL_DATE'], roundTrip_profit_g['outbound_FL_DATE']).dt.month)
                      .groupby('round_trip_route_IATA')
                      .apply(lambda df: df.groupby('month')['profit'].sum().mean())
                      .reset_index()
                      .rename(columns={0:'monthly_avg_profit'}))
roundTrip_profit_m = roundTrip_profit_m[roundTrip_profit_m['round_trip_route_IATA'].isin(roundTrip_profit['round_trip_route_IATA'])]
roundTrip_profit_m = (roundTrip_profit_m.assign(monthly_avg_profit_ranking = roundTrip_profit_m['monthly_avg_profit']
                                                .rank(ascending = False, method = 'min')))

# calculate the weekly average profit
roundTrip_profit_w = (roundTrip_profit_g.assign(week = np.minimum(roundTrip_profit_g['inbound_FL_DATE'], roundTrip_profit_g['outbound_FL_DATE']).dt.isocalendar().week)
                      .groupby('round_trip_route_IATA')
                      .apply(lambda df: df.groupby('week')['profit'].sum().mean())
                      .reset_index()
                      .rename(columns={0:'weekly_avg_profit'}))
roundTrip_profit_w = roundTrip_profit_w[roundTrip_profit_w['round_trip_route_IATA'].isin(roundTrip_profit['round_trip_route_IATA'])]
roundTrip_profit_w = (roundTrip_profit_w.assign(weekly_avg_profit_ranking = roundTrip_profit_w['weekly_avg_profit']
                                                .rank(ascending = False, method = 'min')))

# calculate the ranking of the operation career
roundTrip_op = (roundTrip_profit_g
                .groupby('round_trip_route_IATA')
                .apply(lambda df: len(set(df['inbound_OP_CARRIER']).union(set(df['outbound_OP_CARRIER']))))
                .reset_index()
                .rename(columns={0:'round_trip_op_count'}))

# calcualte the proprotion of the delay flights for each round trip route
inbound_trips = roundTrip_profit_g[['inbound_DEP_DELAY', 'inbound_ARR_DELAY', 'round_trip_route_IATA']]
outbound_trips = roundTrip_profit_g[['outbound_DEP_DELAY', 'outbound_ARR_DELAY', 'round_trip_route_IATA']]

all_flights = pd.concat(
    [inbound_trips.rename(columns={'inbound_DEP_DELAY':'DEP_DELAY', 'inbound_ARR_DELAY':'ARR_DELAY'}, inplace= False),
    outbound_trips.rename(columns={'outbound_DEP_DELAY':'DEP_DELAY', 'outbound_ARR_DELAY':'ARR_DELAY'}, inplace= False)],
    axis=0
)
all_flights = all_flights[all_flights['round_trip_route_IATA'].isin(roundTrip_profit['round_trip_route_IATA'])]

all_flights['is_dep_delay'] = all_flights['DEP_DELAY'].apply(lambda val : True if val >= 15 else False if val <= -15 else 0)
all_flights['is_arr_delay'] = all_flights['ARR_DELAY'].apply(lambda val : True if val >= 15 else False if val <= -15 else 0)

roundTrip_dep_delay_rate = all_flights.groupby('round_trip_route_IATA')['is_dep_delay'].mean().sort_values(ascending=True).reset_index().rename(columns={'is_dep_delay':'dep_delay_rate'})
roundTrip_dep_delay_rate = (roundTrip_dep_delay_rate.assign(dep_delay_rate_ranking = roundTrip_dep_delay_rate['dep_delay_rate'].rank(ascending = True, method = 'min')))

roundTrip_arr_delay_rate = all_flights.groupby('round_trip_route_IATA')['is_arr_delay'].mean().sort_values(ascending=True).reset_index().rename(columns={'is_arr_delay':'arr_delay_rate'})
roundTrip_arr_delay_rate = (roundTrip_arr_delay_rate.assign(arr_delay_rate_ranking = roundTrip_arr_delay_rate['arr_delay_rate'].rank(ascending = True, method = 'min')))

# calcualate the average delay rate for each round trip route



# # merge the ranks
round_trip_info = roundTrip_profit.merge(roundTrip_fre, on='round_trip_route_IATA', how='inner')

round_trip_info = round_trip_info.merge(roundTrip_profit_m, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_profit_w, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_op, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_dep_delay_rate, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_arr_delay_rate, on='round_trip_route_IATA', how='left')




# # # aggregate the ranking
round_trip_info = round_trip_info.assign(avg_ranking = ((round_trip_info['round_trip_flights_count_ranking'] * 0.2 + 
                                                         round_trip_info['round_trip_profit_ranking'] * 0.7 + 
                                                         round_trip_info['monthly_avg_profit_ranking'] * 0.5 +
                                                         round_trip_info['weekly_avg_profit_ranking'] * 0.5) / 4))
candidate_roundTrip_route = round_trip_info.sort_values(by='avg_ranking', ascending=True).head(10)

In [3]:
candidate_roundTrip_route['round_trip_route_IATA'].to_csv('../data/temproary_data/candidate_roundTrip_route.csv', index=False)